In [1]:
import sys
!{sys.executable} -m pip install spacy

In [2]:
import import_ipynb
import os
import pickle
import numpy as np
import spacy
from spacy import ""
from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

print(spacy.__version__)

2.2.3


In [3]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("db.sqlite3")
df = pd.read_sql_query("select * from submissions limit 5000;", conn)


In [4]:
df.head(10)

,subreddit,subreddit_subs,title,text
0,AskReddit,26256285,"People who haven't pooped in 2019 yet, why are...",
1,AskReddit,26256285,Stan Lee has passed away at 95 years old,As many of you know today is day that many of ...
2,Home,10068,Beautiful Home :),
3,Home,10068,This was finished yesterday..,
4,videos,22083867,This is what happens when one company owns doz...,
5,videos,22083867,Youtube is Facilitating the Sexual Exploitatio...,
6,news,19783893,Blizzard Employees Staged a Walkout After the ...,
7,news,19783893,Kobe Bryant killed in helicopter crash in Cali...,
8,worldnews,23061646,Trump Impeached for Abuse of Power,
9,worldnews,23061646,"Two weeks before his inauguration, Donald J. T...",


In [5]:
df['text'].head(5)

0                                                     
1    As many of you know today is day that many of ...
2                                                     
3                                                     
4                                                     
Name: text, dtype: object

In [6]:
df['clean_text'] = df['text'].str.replace('[^\w\s]',' ')
df['clean_title'] = df['title'].str.replace('[^\w\s]',' ')

In [7]:
df.head(10)

,subreddit,subreddit_subs,title,text,clean_text,clean_title
0,AskReddit,26256285,"People who haven't pooped in 2019 yet, why are...",,,People who haven t pooped in 2019 yet why are...
1,AskReddit,26256285,Stan Lee has passed away at 95 years old,As many of you know today is day that many of ...,As many of you know today is day that many of ...,Stan Lee has passed away at 95 years old
2,Home,10068,Beautiful Home :),,,Beautiful Home
3,Home,10068,This was finished yesterday..,,,This was finished yesterday
4,videos,22083867,This is what happens when one company owns doz...,,,This is what happens when one company owns doz...
5,videos,22083867,Youtube is Facilitating the Sexual Exploitatio...,,,Youtube is Facilitating the Sexual Exploitatio...
6,news,19783893,Blizzard Employees Staged a Walkout After the ...,,,Blizzard Employees Staged a Walkout After the ...
7,news,19783893,Kobe Bryant killed in helicopter crash in Cali...,,,Kobe Bryant killed in helicopter crash in Cali...
8,worldnews,23061646,Trump Impeached for Abuse of Power,,,Trump Impeached for Abuse of Power
9,worldnews,23061646,"Two weeks before his inauguration, Donald J. T...",,,Two weeks before his inauguration Donald J T...


In [8]:
# We set our features as description, and target as subreddit.  
# Create a mass text.

features = ['clean_text', 'clean_title', 'subreddit_subs'] 
target = 'subreddit'

X = df[features]
y = df[[target]]

In [9]:
#Create the nlp object
nlp = spacy.load("en_core_web_md")

# create tokenizer object
tokenizer = Tokenizer(nlp.vocab)

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
def tokenize(doc):
        """Return the tokens"""
        return [token.text for token in tokenizer(doc)]

In [ ]:
def get_lemmas(text):
        """Return the Lemmas"""
        lemmas = []
        doc = nlp(text)
    
        for token in doc: 
            if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
                lemmas.append(token.lemma_)
    
        return lemmas

In [ ]:
text = df["clean_text"]

# Instantiate vectorizer object
#tfidf = TfidfVectorizer(tokenizer=get_lemmas, min_df=0.025, max_df=.98, ngram_range=(1,2))
tfidf = TfidfVectorizer(stop_words="english")
# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(text) # Similiar to fit_predict

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [ ]:
# Fit on TF-IDF Vectors
size = 5
nn  = NearestNeighbors(n_neighbors=size, algorithm='ball_tree')
nn.fit(dtm)

In [ ]:
user_input = ["""Capes are not a great idea of you are a super hero trying to save the world."""]

vec_user_input = tfidf.transform(user_input)
dist, subreddit_index = nn.kneighbors(vec_user_input.todense())

In [ ]:
recommended_reddits = [df[['subreddit','title','text','subscribers']].iloc[n] for n in subreddit_index]

print(recommended_reddits)

In [ ]:
#do pickling here